### This notebook shows some lower-level functionality in `flopy` for working with shapefiles
including:
* `recarray2shp` convience function for writing a numpy record array to a shapefile
* `shp2recarray` convience function for quickly reading a shapefile into a numpy recarray
* `utils.geometry` classes for writing shapefiles of model input/output. For example, quickly writing a shapefile of model cells with errors identified by the checker
* demonstration of how the `epsgRef` class works for retrieving projection file information (WKT text) from spatialreference.org, and caching that information locally for when an internet connection isn't available
* how to reset `epsgRef` if it becomes corrupted
* examples of how the `Point` and `LineString` classes can be used to quickly plot pathlines and endpoints from MODPATH (these are also used by the `PathlineFile` and `EndpointFile` classes to write shapefiles of this output)

In [ ]:
%matplotlib inline
import os
import sys

# python < 3.4 (reload in default namespace)
try:
    from importlib import reload
except:
    from imp import reload

import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy
    
from flopy.utils.geometry import Polygon, LineString, Point
from flopy.export.shapefile_utils import recarray2shp, shp2recarray
from flopy.utils.modpathfile import PathlineFile, EndpointFile
from flopy.utils import geometry
from flopy.utils.reference import epsgRef
ep = epsgRef()
ep.reset()

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

### write a numpy record array to a shapefile
in this case, we want to visualize output from the checker  
first make a toy model

In [ ]:
m = flopy.modflow.Modflow('toy_model', model_ws='data')
botm = np.zeros((2, 10, 10))
botm[0, :, :] = 1.5
botm[1, 5, 5] = 4 # negative layer thickness!
botm[1, 6, 6] = 4
dis = flopy.modflow.ModflowDis(nrow=10, ncol=10, 
                               nlay=2, delr=100, delc=100,
                               top=3, botm=botm, model=m)

In [ ]:
grid = m.modelgrid
grid.set_coord_info(xoff=600000, yoff=5170000, proj4='EPSG:26715', angrot=45)

In [ ]:
chk = dis.check()
chk.summary_array

### make geometry objects for the cells with errors  
*  geometry objects allow the shapefile writer to be simpler and agnostic about the kind of geometry

In [ ]:
get_vertices = m.modelgrid.get_cell_vertices # function to get the referenced vertices for a model cell
geoms = [Polygon(get_vertices(i, j)) for i, j in chk.summary_array[['i', 'j']]]

In [ ]:
geoms[0].type

In [ ]:
geoms[0].exterior

In [ ]:
geoms[0].bounds

In [ ]:
geoms[0].plot() # this feature requires descartes

### write the shapefile  
* the projection (.prj) file can be written using an epsg code  
* or copied from an existing .prj file

In [ ]:
recarray2shp(chk.summary_array, geoms, 'data/test.shp', epsg=26715)

In [ ]:
shutil.copy('data/test.prj', 'data/26715.prj')
recarray2shp(chk.summary_array, geoms, 'data/test.shp', prj='data/26715.prj')

### read it back in  
* flopy geometry objects representing the shapes are stored in the 'geometry' field

In [ ]:
ra = shp2recarray('data/test.shp')
ra

In [ ]:
ra.geometry[0].plot()

### How the epsg feature works  
* requires an internet connection the first time to get the prj text from [spatialreference.org](http://spatialreference.org/) using ```requests```  
* if it doesn't exist, ```epsgref.py``` is created in the site-packages folder  
* the prj text is then stashed in this file as an entry in a dictionary ```prj```  
* this dictionary is imported and checked for the prjtext each time flopy.utils.reference.getprj is called

In [ ]:
import epsgref
reload(epsgref)
from epsgref import prj
prj

In [ ]:
from flopy.utils.reference import getprj, epsgRef

In [ ]:
getprj(4326)

In [ ]:
reload(epsgref)
from epsgref import prj
for k, v in prj.items():
    print('{}:\n{}\n'.format(k, v))

### working with the ```flopy.utils.reference.epsgRef``` handler

In [ ]:
ep = epsgRef()

In [ ]:
ep.add(9999, 'junk')

In [ ]:
epsgRef.show()

#### remove an entry

In [ ]:
ep.remove(9999)
epsgRef.show()

#### start over with a new file

In [ ]:
ep.reset()

In [ ]:
reload(epsgref)
from epsgref import prj
prj

In [ ]:
len(prj.keys())

## Other geometry types

### Linestring  
* create geometry objects for pathlines from a MODPATH simulation
* plot the paths using the built in plotting method

In [ ]:
pthfile = PathlineFile('../data/mp6/EXAMPLE-3.pathline')
pthdata = pthfile._data.view(np.recarray)

In [ ]:
length_mult = 1. # multiplier to convert coordinates from model to real world
rot = 0 # grid rotation

particles = np.unique(pthdata.particleid)
geoms = []
for pid in particles:
    ra = pthdata[pthdata.particleid == pid]

    x, y = geometry.rotate(ra.x * length_mult,
                           ra.y * length_mult,
                           grid.xoffset,
                           grid.yoffset,
                           rot)
    z = ra.z
    geoms.append(LineString(list(zip(x, y, z))))

In [ ]:
geoms[0]

In [ ]:
geoms[0].plot()

In [ ]:
fig, ax = plt.subplots()
for g in geoms:
    g.plot(ax=ax)
ax.autoscale()
ax.set_aspect(1)

## Points

In [ ]:
eptfile = EndpointFile('../data/mp6/EXAMPLE-3.endpoint')
eptdata = eptfile.get_alldata()

In [ ]:
x, y = geometry.rotate(eptdata['x0'] * length_mult,
                       eptdata['y0'] * length_mult,
                       grid.xoffset,
                       grid.yoffset,
                       rot)
z = eptdata['z0']

geoms = [Point(x[i], y[i], z[i]) for i in range(len(eptdata))]

In [ ]:
fig, ax = plt.subplots()
for g in geoms:
    g.plot(ax=ax)
ax.autoscale()
ax.set_aspect(2e-6)